In [ ]:
!pip install -q diffusers transformers accelerate openai

### Load the model

In [ ]:
import torch
from diffusers import DiffusionPipeline
import openai
import json

device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "youngmki/musinsaigo-2.0"
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)
pipe = pipe.to(device)
pipe.load_lora_weights(model_id)

Prompt helper functions

In [ ]:
def make_prompt(prompt: str) -> str:
    prompt_prefix = "RAW photo"
    prompt_suffix = "(high detailed skin:1.2), 8k uhd, dslr, soft lighting, high quality, film grain, Fujifilm XT3"
    return ", ".join([prompt_prefix, prompt, prompt_suffix]).strip()


def make_negative_prompt(negative_prompt: str) -> str:
    negative_prefix = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), \
    text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, \
    extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, \
    bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, \
    extra arms, extra legs, fused fingers, too many fingers, long neck"

    return (
        ", ".join([negative_prefix, negative_prompt]).strip()
        if len(negative_prompt) > 0
        else negative_prefix
    )

Inference

In [ ]:
PROMPT = "A Korean woman in her 20s wearing a white large hoodie in street fashion"
NEGATIVE_PROMPT = ""

def generate_image(prompt):
    image = pipe(
        prompt=make_prompt(prompt),
        height=1024,
        width=768,
        num_inference_steps=50,
        guidance_scale=7.5,
        negative_prompt=make_negative_prompt(NEGATIVE_PROMPT),
        cross_attention_kwargs={"scale": 0.75},
    ).images[0]

    return image

generate_image(PROMPT)

ChatGPT API

In [ ]:
import openai
import json

openai.api_key = "aaaaa"

1. 조건이 부족할 때

In [ ]:
gpt_system_prompt = """You're a chatbot that helps people choose what to wear today. Based on the user's mood and preferences, you need to help them choose the right outfit and print it out as an image. To make outfit recommendations, you need the following information.
- The user's gender
- Nationality
- Age
- Mood
- Clothing preferences

To create a prompt to generate an image, the process is as follows:
1. Collect information from the user by asking them a number of questions.
2. Once you have enough information, complete the prompt as shown below:
Example input:
- Gender: Female
- Nationality: Korea
- Age: 25
- Mood: Fresh
- Preference: Street fashion
Example prompt: A Korean woman in her 20s wearing a white large hoodie in street fashion.
3. pass the completed prompt as an argument to call the `generate_image` function.
4. You must answer in Korean and prompt must be in English.
"""

gpt_user_prompt = "오늘 어떤 옷을 입어야할까?"

completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[
        {"role": "system", "content": gpt_system_prompt},
        {"role": "user", "content": gpt_user_prompt}],
    functions=[{
        "name": "generate_image",
        "description": "Generate an outfit image from prompt.",
        "parameters": {
            "type": "object",
            "properties": {
                "prompt": {
                    "type": "string",
                    "description": "Prompt that generate an outfit image",
                },
            },
            "required": ["prompt"],
        },
    }],
    function_call="auto",
)

print(completion.choices[0])
print(completion.choices[0].message.content)

2. 조건을 만족했을 때

In [ ]:
gpt_user_prompt = "오늘 어떤 옷을 입어야할까? 나이: 25세, 국적: 대한민국, 성별: 여성, 기분: 우울함, 취향: 단정함"

completion = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages=[
        {"role": "system", "content": gpt_system_prompt},
        {"role": "user", "content": gpt_user_prompt}],
    functions=[{
        "name": "generate_image",
        "description": "Generate an outfit image from prompt.",
        "parameters": {
            "type": "object",
            "properties": {
                "prompt": {
                    "type": "string",
                    "description": "Prompt that generate an outfit image",
                },
            },
            "required": ["prompt"],
        },
    }],
    function_call="auto",
)

print(completion.choices[0])
print(completion.choices[0].message.content)

Result

In [ ]:
if "function_call" in completion.choices[0].message:
    function_name = completion.choices[0].message.to_dict()['function_call']['name']
    arguments = completion.choices[0].message.to_dict()['function_call']['arguments']
    arguments = json.loads(arguments)

    if function_name == "generate_image":
        print(function_name)
        print(arguments)

Gradio Chatbot

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

def answer(state, state_chatbot, text):
    messages = state + [{
        "role": "user",
        "content": text
    }]

    completion = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=[{
            "name": "generate_image",
            "description": "Generate an outfit image from prompt.",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "Prompt that generate an outfit image",
                    },
                },
                "required": ["prompt"],
            },
        }],
        function_call="auto",
    )

    msg = completion.choices[0].message.content if completion.choices[0].message.content else ""

    new_state = [{
        "role": "user",
        "content": text
    }, {
        "role": "assistant",
        "content": msg
    }]

    state = state + new_state

    if "function_call" in completion.choices[0].message:
        function_name = completion.choices[0].message.to_dict()['function_call']['name']
        arguments = completion.choices[0].message.to_dict()['function_call']['arguments']
        arguments = json.loads(arguments)

        if function_name == "generate_image":
            print(arguments)

            img = generate_image(arguments["prompt"])

            img_path = f'image.jpg'
            img.save(img_path)

            state_chatbot = state_chatbot + [(text, f'{msg}\n![](/file={img_path})')]
    else:
        state_chatbot = state_chatbot + [(text, msg)]

    return state, state_chatbot, state_chatbot


with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    state = gr.State([{
        "role": "system",
        "content": gpt_system_prompt
    }])
    state_chatbot = gr.State([])

    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>OOTD Recommendation Chatbot</h1>
            </div>
            <p style="margin-bottom: 10px; font-size: 94%">
                YouTube <a href="https://www.youtube.com/@bbanghyong">빵형의 개발도상국</a>
            </p>
        </div>""")

    with gr.Row():
        chatbot = gr.Chatbot(elem_id="chatbot")

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="오늘 무슨 옷을 입으면 좋을까?", container=False)

    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: "" , None, txt)


demo.launch(debug=True, share=True)